# Trip Distribution Class

# Data
## The result from trip generation (trip production and attraction) will be passed to trip distribution

In [1]:

from IPython.display import HTML, display
import math as math


class TripDistribution:

    def __init__(self, productions, attractions, travelTime, fare, income):
        self.productions = productions
        self.attractions = attractions
        self.travelTime = travelTime
        self.fare = fare
        self.income = income
        self.row = len(productions)
        self.col = len(attractions)
        self.possibleError = sum(productions) * 0.2
        self.error = 0

    def getGeneralizedCost(self, cost):
        return 1.0 / (cost * cost)
    
    def computeCost(self, travelTime, fare, income):
        costMatrix = [[1 for x in range(self.row)] for y in range(self.col)]
        for x in range(self.row):
            for y in range(self.col):
                costMatrix[x][y] = travelTime[x][y] * income[x] + fare[x][y]
        print(costMatrix)
        return costMatrix

    def getTripDistribution(self):
        distributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        #costMatrix = [[1 for x in range(self.row)] for y in range(self.col)]
        costMatrix = self.computeCost(self.travelTime, self.fare, self.income)
        A = [1 for x in range(self.row)]
        B = [1 for x in range(self.col)]

        currentBalancingFactor = 0  # 0 for A, 1 for B
        isConvergent = False
        shit = False

        while isConvergent == False:
            if currentBalancingFactor == 0:
                tempA = self.computeA(B, costMatrix)
                A = tempA
                currentBalancingFactor = 1
            elif currentBalancingFactor == 1:
                tempB = self.computeB(A, costMatrix)
                B = tempB
                currentBalancingFactor = 0
            distributions = self.computeDistributions(A, B, costMatrix)
            isConvergent = self.checkIfConvergent(distributions)
            print(str(A) + " " + str(B))
        return distributions

    def computeDistributions(self, A, B, costMatrix):
        distributions = [[self.attractions[y] for x in range(self.row)] for y in range(self.col)]
        for x in range(self.row):
            for y in range(self.col):
                distributions[x][y] = A[x] * self.productions[x] * B[y] * self.attractions[y] * self.getGeneralizedCost(costMatrix[x][y])
        return distributions

    def checkIfConvergent(self, distributions):
        error = self.getError(distributions)
        if error <= self.possibleError:
            self.error = error
            return True
        return False

    def getError(self, distributions):
        error = 0
        derivedProductions = [0 for x in range(self.row)]
        derivedAttractions = [0 for x in range(self.col)]

        for x in range(self.row):
            for y in range(self.col):
                derivedProductions[x] += distributions[x][y]
                derivedAttractions[y] += distributions[x][y]

        for x in range(self.row):
            error += abs(derivedProductions[x] - self.productions[x])
            error += abs(derivedAttractions[y] - self.attractions[y])

        return error
    
    
    

    def computeA(self, B, costMatrix):
        A = [1 for x in range(self.row)]
        for x in range(0, self.row):
            sum = 0.0
            for y in range(0, self.col):
                sum += B[y] * self.attractions[y] * self.getGeneralizedCost(costMatrix[x][y])
            A[x] = 1.0 / sum
        return A

    def computeB(self, A, costMatrix):
        B = [1 for x in range(self.col)]
        for x in range(0, self.row):
            sum = 0.0
            for y in range(0, self.col):
                sum += A[y] * self.productions[y] * self.getGeneralizedCost(costMatrix[x][y])
            B[x] = 1.0 / sum
        return B

In [2]:
def computeYearlyToHourlyRate(salary):
    return salary/(30.00 * 8)

def deg2rad(deg):
    return deg * (math.pi/180)

def getDistance(lat1, lng1, lat2, lng2):
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    a = ((math.sin(deg2rad(dlat/2)))*(math.sin(deg2rad(dlat/2)))) + math.cos(deg2rad(lat1)) *math.cos(deg2rad(lat2)) *((math.sin(deg2rad(dlon/2)))*(math.sin(deg2rad(dlon/2))))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d =  6373* c
    return d



def computeNearestZones(lat, lng) :
    nearestZones = [None] * len(lat)
    for currZone in range(len(lat)):
        shortest = 1000
        nearestZoneIndex = 0
        for currPoint in range(len(lat)):
            if(currZone != currPoint): 
                distance = getDistance(lat[currZone], lng[currZone], lat[currPoint], lng[currPoint])
                if(distance < shortest):
                    print(distance)
                    shortest = distance
                    nearestZoneIndex = currPoint
        nearestZones[currZone] = nearestZoneIndex
        shortest = 1000
        nearestZoneIndex = 0
    return nearestZones

productions = [
    167690.52,
    108791.67,
    126709.77,
    27078.36, 
    50512.34, 
    65431.76, 
    26142.93, 
    152812.1
]
attractions = [
    97800.711, 
    91560.71, 
    92589.09, 
    84274.9, 
    84274.9, 
    93634.9, 
    88440.71, 
    92589.09
]

nearestZones = [
    0, 
    0,
    0,
    0,
    0,
    0,
    0,
    0
]

lat = [
    14.711029630751996,
    14.681442858628275,
    14.708693067291241, 
    14.686174778272566, 
    14.660783607519472,
    14.65615255044289,
    14.630416868404804,
    14.637488044822764
]

lng = [
    120.960596201819,
    120.98000062117379,
    121.00151327979542,
    121.00714359391715,
    121.09957600978247,
    121.11920195444038,
    121.080185938878,
    121.09795399889786
]

travelTime = [
        [001.00, 027.00, 053.67, 056.67, 140.67, 142.67, 114.67, 123.67], 
        [028.33, 001.00, 050.67, 051.00, 124.33, 132.00, 098.33, 106.67],
        [052.33, 050.33, 001.00, 037.00, 127.67, 143.00, 109.00, 117.33], 
        [057.00, 051.00, 037.00, 001.00, 122.67, 130.00, 096.33, 104.67],
        [151.33, 133.67, 136.00, 131.33, 001.00, 031.67, 038.67, 026.67],
        [157.00, 139.33, 149.67, 137.33, 032.00, 001.00, 041.67, 030.33],
        [120.00, 102.00, 113.00, 096.67, 037.00, 040.67, 001.00, 015.00], 
        [129.00, 111.00, 121.67, 105.33, 029.00, 029.67, 015.00, 001.00]
        ]

fares = [
        [01.00, 10.67, 17.65, 25.33, 54.35, 53.55, 47.00, 48.65], 
        [12.00, 01.00, 56.65, 16.37, 45.60, 45.14, 38.58, 40.23],
        [17.65, 16.22, 01.00, 10.67, 43.72, 46.08, 40.25, 40.57], 
        [16.67, 18.67, 10.67, 01.00, 42.02, 41.64, 35.08, 36.73],
        [52.00, 43.33, 40.97, 39.41, 01.00, 10.67, 11.31, 08.00], 
        [52.77, 44.10, 45.35, 40.27, 08.00, 01.00, 10.64, 08.03],
        [45.17, 36.50, 37.83, 34.50, 11.01, 10.24, 01.00, 08.00], 
        [47.43, 38.76, 40.09, 36.03, 09.33, 08.00, 08.00, 01.00]
        ]
salary = [computeYearlyToHourlyRate(24570.49), 
          computeYearlyToHourlyRate(16576.02), 
          computeYearlyToHourlyRate(21038.92), 
          computeYearlyToHourlyRate(16985.72), 
          computeYearlyToHourlyRate(29038.00), 
          computeYearlyToHourlyRate(30548.05), 
          computeYearlyToHourlyRate(27276.91), 
          computeYearlyToHourlyRate(39221.55)];
            
        

    
nearestZones = computeNearestZones(lat, lng)

for x in range(len(productions)): 
    fares[x][x] = fares[x][nearestZones[x]] / 2
    travelTime[x][x] = travelTime[x][nearestZones[x]] / 2

td = TripDistribution(productions, attractions, travelTime, fares, salary)

print("Productions: " + str(productions))
print("Attractions: " + str(attractions))
print(fares)
print(travelTime)

3.8972919033706943
3.8972919033706943
3.8137190579442217
2.9675509846852295
4.409695353064177
3.8137190579442217
2.576917469557242
5.7204275656322325
2.9675509846852295
2.576917469557242
15.963942541851932
13.070287911910077
11.820569829043027
10.3391369702715
2.173848992699761
18.12430372972955
15.240532763981337
13.946521878486973
12.511710142114591
2.173848992699761
15.684105726598903
12.183710760129195
12.143735071696549
10.01221924018903
3.9702638544076576
2.0676624512310244
16.89257423199684
13.601889082590215
13.054326989667869
11.172150066414227
2.5970341357325517
2.0676624512310244
Productions: [167690.52, 108791.67, 126709.77, 27078.36, 50512.34, 65431.76, 26142.93, 152812.1]
Attractions: [97800.711, 91560.71, 92589.09, 84274.9, 84274.9, 93634.9, 88440.71, 92589.09]
[[5.335, 10.67, 17.65, 25.33, 54.35, 53.55, 47.0, 48.65], [12.0, 8.185, 56.65, 16.37, 45.6, 45.14, 38.58, 40.23], [17.65, 16.22, 5.335, 10.67, 43.72, 46.08, 40.25, 40.57], [16.67, 18.67, 10.67, 5.335, 42.02, 41.64

In [3]:
distribution = td.getTripDistribution()

display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in distribution)
        )
))

[[1387.4250625000002, 2774.8501250000004, 5512.22582625, 5827.03695125, 14455.72845125, 14659.682534583331, 11786.575367916666, 12709.618742916668], [1968.6610274999998, 1769.387125, 3556.2622225, 3538.77425, 8632.6690275, 9161.951, 6829.9135275, 7407.5802225], [4605.011181666666, 4428.256848333333, 1627.0850833333334, 3254.170166666667, 11235.548818333333, 12581.769833333334, 9595.426166666666, 10325.972015], [4050.778500000001, 3628.135500000001, 2629.301833333334, 1314.650916666667, 8723.846135000002, 9242.238333333335, 6852.723365000001, 7444.627135000001], [18361.668916666666, 16216.28608333333, 16495.836666666666, 15929.245583333333, 1921.2380416666667, 3842.4760833333335, 4690.05775, 3234.84775], [20036.286041666666, 17778.515860416668, 19095.877681249996, 17520.118777083335, 4081.0733333333333, 2040.5366666666666, 5314.5451812500005, 3868.53981875], [13683.625, 11629.186749999999, 12880.708458333333, 11021.412040416666, 4216.200291666667, 4632.539707083333, 856.4034375, 1712.80

105308.14613995228,29646.934872718703,10502.31331259198,11277.454813381226,3153.667638992995,3646.5630590847554,2524.410566586952,1631.029596691112
26901.054126982504,37501.03755831734,12977.217055073235,15726.463484894557,4548.161225589207,4801.6121550873495,3866.65124277439,2469.473151281437
6232.339223467562,7589.7012137685915,78586.84795455726,23575.317216419015,3403.6016552887413,3227.608203451359,2483.351254881677,1611.0032781657774
1021.5467557650207,1433.9898191993511,3816.9132384524232,18320.610682560775,716.0347398035177,758.6347550036011,617.5371187673396,393.09289044797003
109.68531567322928,158.36084932379387,213.93509470925054,275.3011210847001,32570.56451032479,9682.829148563042,2908.5105251403047,4593.153435180883
126.51587499061384,180.95320760534065,219.25936737937852,312.55825901802916,9913.92247969113,47156.59995761758,3111.010586943475,4410.940266754473
43.58278196036647,67.95108721135465,77.42758442261993,126.90198048188081,1492.415169424542,1470.0455646312612,19249.289268667264,3615.3165632007117
232.9166007373843,354.3315001254222,412.3568929249809,660.1415989463078,14991.408031410907,17041.847206594695,29742.229499175126,89376.8686700852


# Error


The following formula, which is from a book, Introduction to Transportation Engineering by Tom V. Mathew and K V Krishna Rao, is used:

$$T_{ij} = A_iO_iB_jD_jf(c_{ij})$$

Where $T_{ij}$ is the trips made from zone i to the zone j, $O_i$ refers to the trip productions made from zone i, $D_j$ refers to the trip attractions made from zone j, $c_{ij}$ is the general cost used from zone i to zone j, $A_i$ and $B_i$ are the balancing factor and the following formula are used:

 $$A_i = 1 / \sum_{j} B_{j}D_jf(c_{ij})$$
 $$B_i = 1 / \sum_{i} A_{i}O_if(c_{ij})$$
 
At first, one of the balancing factors will be set to 1s and the other will be compupted using the formula above. This will be done alternately until the some of all the origins or the destinations of the computed distributions will be close to the original productions and attractions. Lastly, the function $f(c_{ij})$ has the following formula:
$$f(c_{ij}) = 1/c_{ij}^2$$

$$c_{ij} = t_{ij}s_i + f_{ij}$$



In [4]:
print("Error: " + str(td.error))

Error: 124094.30281446071
